In [1]:
import requests
from bs4 import BeautifulSoup as bs
import urllib.request
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import numpy as np
import time
import re
def nans(df): return df[df.isnull().any(axis=1)]

In [2]:
#Generating List of URL's
audley_end = {'audleyend':'71589'}
team_ids = {'audleyend':'71589', 'ncissc':'9822', 'gchishill':'63027', 'lintonvillage':'9962',
        'ickleton':'165509', 'birchanger':'85067', 'elmdoncc':'51838', 'littleshelford':'19931',
        'ashwell':'52157'}
years = ['2017', '2018', '2019', '2021']
urls = []
def genurl(team_dict, years):
    for year in years:
        for key, value in team_dict.items():
            url = f'https://{key}.play-cricket.com/website/team_matches/{value}'
            urls.append(url)
print(urls)
            
            
        
    

[]


In [3]:
genurl(team_ids, years)
print(urls)

['https://audleyend.play-cricket.com/website/team_matches/71589', 'https://ncissc.play-cricket.com/website/team_matches/9822', 'https://gchishill.play-cricket.com/website/team_matches/63027', 'https://lintonvillage.play-cricket.com/website/team_matches/9962', 'https://ickleton.play-cricket.com/website/team_matches/165509', 'https://birchanger.play-cricket.com/website/team_matches/85067', 'https://elmdoncc.play-cricket.com/website/team_matches/51838', 'https://littleshelford.play-cricket.com/website/team_matches/19931', 'https://ashwell.play-cricket.com/website/team_matches/52157', 'https://audleyend.play-cricket.com/website/team_matches/71589', 'https://ncissc.play-cricket.com/website/team_matches/9822', 'https://gchishill.play-cricket.com/website/team_matches/63027', 'https://lintonvillage.play-cricket.com/website/team_matches/9962', 'https://ickleton.play-cricket.com/website/team_matches/165509', 'https://birchanger.play-cricket.com/website/team_matches/85067', 'https://elmdoncc.play

In [4]:
short_urls = urls[:3]
tiny_urls = urls[:1]
short_urls
tiny_urls

['https://audleyend.play-cricket.com/website/team_matches/71589']

In [166]:
home_team_list = []
home_team_score_list = []
home_team_wickets_lost_list = []
away_team_list = []
away_team_score_list = []
away_team_wickets_lost_list = []
result_list = []
years = []
winner = []
bat_first_win = []
bat_second_win = []

for url in short_urls:
    s=Service('/Users/MatthewBeale/Desktop/chromedriver')
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    time.sleep(2)
    xpath_cookies = r'/html/body/div[11]/div/div/div/div[1]/div[2]/div/button'
    driver.find_element(By.XPATH, xpath_cookies).click()
    cricketsoup = bs(driver.page_source, 'html.parser')
    driver.close()
    cricket_data = cricketsoup.find_all('div', class_ = 'container section-results')
    #Finding teams and dividing into home and away teams
    all_teams_list = []
    for line in cricket_data:
        home_teams = line.find_all('p', class_='txt1')
        for team in home_teams:
            team = team.text.upper()
            team = team.split(' ')
            team = team[:-3]
            team = ' '.join(team)
            all_teams_list.append(team)
    home_team_list_prep = []
    home_team_list_prep.append(all_teams_list[::2])
    for item in home_team_list_prep:
        for element in item:
            home_team_list.append(element)
    away_team_list_prep = []
    away_team_list_prep.append(all_teams_list[1::2])
    for item in away_team_list_prep:
        for element in item:
            away_team_list.append(element)
    #Finding scores
    for line in cricket_data:
        scores = line.find_all('p', class_='txt2 hs')
        all_runs = []
        for score in scores:
            runs = score.text.strip()
            runs = runs.split('/')[0].strip()
            all_runs.append(runs)
        home_runs_prep = []
        home_runs_prep.append(all_runs[::2])
        for lst in home_runs_prep:
            for item in lst:
                home_team_score_list.append(item)
        away_runs_prep = []
        away_runs_prep.append(all_runs[1::2])
        for lst in away_runs_prep:
            for item in lst:
                away_team_score_list.append(item)
    #Finding result and year
    for line in cricket_data:
        results = line.find_all('div', class_ = 'fonts-gt')
        for result in results:
            result_list.append(result.text.upper())
            year = url.split('?')[1]
            year = year.split('=')[1]
            year = year.split('&')[0]
            years.append(year)
            win = result.text.split('WON')[0].strip()
            if win == 'ABANDONED':
                win = None
            elif win == 'CANCELLED':
                win = None
            elif 'CONCEDED' in win:
                win = None
            else:
                win = win
            winner.append(win)
            margin = result.text.split(' ')[-1].strip()
            #print(margin)
            if margin == 'WICKETS':
                bat_first_win.append(False)
                bat_second_win.append(True)
            elif margin == 'WICKET':
                bat_first_win.append(False)
                bat_second_win.append(True)
            elif margin == 'RUN':
                bat_first_win.append(True)
                bat_second_win.append(False)
            elif margin == 'RUNS':
                bat_first_win.append(True)
                bat_second_win.append(False)
            elif margin == 'ABANDONED':
                bat_first_win.append(None)
                bat_second_win.append(None)
            elif margin == 'CANCELLED':
                bat_first_win.append(None)
                bat_second_win.append(None)
            elif margin == 'CONCEDED':
                bat_first_win.append(None)
                bat_second_win.append(None)
    #Finding wickets taken
    for line in cricket_data:
        scores = line.find_all('p', class_ = 'txt2 hs')
        wickets_prep_list = []
        home_wickets_prep_list = []
        away_wickets_prep_list = []
        for score in scores:
            wickets = score.text.strip()
            wickets = wickets.split('/')
            if wickets != ['']:
                wickets = wickets[1].split('  ')[0].strip()
                if wickets == 'All out':
                    wickets = 10
            else:
                wickets = None
            wickets_prep_list.append(wickets)
    home_wickets_prep_list = wickets_prep_list[::2]
    for item in home_wickets_prep_list:
        home_team_wickets_lost_list.append(item)
    away_wickets_prep_list = wickets_prep_list[1::2]
    for item in away_wickets_prep_list:
        away_team_wickets_lost_list.append(item)
        
#print((home_team_score_list))
#print((away_team_score_list))
#print((home_team_list))
#print((away_team_list))
#print((result_list))
#print((home_team_wickets_lost_list))
#print((away_team_wickets_lost_list))
#print(years)
#print(winner)   
print(len(bat_first_win))
print(len(bat_second_win))
    


52
52


In [181]:
cricketdf = pd.DataFrame({'Year':years, 'Home_Team':home_team_list, 'Home_Score':home_team_score_list,
                          'Home_Team_wkts_lost':home_team_wickets_lost_list,
                          'Away_Team':away_team_list, 'Away_Score':away_team_score_list,
                          'Away_Team_wkts_lost':away_team_wickets_lost_list,
                          'Result':result_list, 'Winner':winner, 'Bat_First_Win':bat_first_win, 
                          'Bat_Second_Win':bat_second_win})
cricketdf['Home_Team_Win'] = np.where(cricketdf['Home_Team']==cricketdf['Winner'], True, False)
cricketdf['Away_Team_Win'] = np.where(cricketdf['Away_Team']==cricketdf['Winner'], True, False)
cricketdf

,Year,Home_Team,Home_Score,Home_Team_wkts_lost,Away_Team,Away_Score,Away_Team_wkts_lost,Result,Winner,Bat_First_Win,Bat_Second_Win,Home_Team_Win,Away_Team_Win
0,2017,AUDLEY END & LITTLEBURY CC,181,7,"BARRINGTON CC, CAMBS",180,8,AUDLEY END & LITTLEBURY CC WON BY 3 WICKETS,AUDLEY END & LITTLEBURY CC,False,True,True,False
1,2017,SAWSTON AND BABRAHAM CC,154,8,AUDLEY END & LITTLEBURY CC,155,9,AUDLEY END & LITTLEBURY CC WON BY 1 WICKET,AUDLEY END & LITTLEBURY CC,False,True,False,True
2,2017,AUDLEY END & LITTLEBURY CC,213,7,MADINGLEY CC,201,9,AUDLEY END & LITTLEBURY CC WON BY 12 RUNS,AUDLEY END & LITTLEBURY CC,True,False,True,False
3,2017,AUDLEY END & LITTLEBURY CC,208,9,YELLING CC,120,10,AUDLEY END & LITTLEBURY CC WON BY 88 RUNS,AUDLEY END & LITTLEBURY CC,True,False,True,False
4,2017,CHESTERFORDS CC,145,7,AUDLEY END & LITTLEBURY CC,147,2,AUDLEY END & LITTLEBURY CC WON BY 8 WICKETS,AUDLEY END & LITTLEBURY CC,False,True,False,True
5,2017,AUDLEY END & LITTLEBURY CC,124,2,WILBRAHAMS CC,122,5,AUDLEY END & LITTLEBURY CC WON BY 8 WICKETS,AUDLEY END & LITTLEBURY CC,False,True,True,False
6,2017,AUDLEY END & LITTLEBURY CC,100,3,ASHWELL CC,,None,ABANDONED,None,None,None,False,False
7,2017,MADINGLEY CC,227,7,AUDLEY END & LITTLEBURY CC,72,10,MADINGLEY CC WON BY 155 RUNS,MADINGLEY CC,True,False,True,False
8,2017,AUDLEY END & LITTLEBURY CC,172,10,THRIPLOW CC,238,7,THRIPLOW CC WON BY 66 RUNS,THRIPLOW CC,True,False,False,True
9,2017,WILBRAHAMS CC,161,6,AUDLEY END & LITTLEBURY CC,158,10,WILBRAHAMS CC WON BY 4 WICKETS,WILBRAHAMS CC,False,True,True,False


In [194]:
home_team_list = []
home_team_score_list = []
home_team_wickets_lost_list = []
away_team_list = []
away_team_score_list = []
away_team_wickets_lost_list = []
result_list = []
years = []
winner = []
bat_first_win = []
bat_second_win = []

for url in urls:
    page = requests.get(url)
    cricketsoup = bs(page.text, 'html.parser')
    cricket_data = cricketsoup.find_all('div', class_ = 'container section-results')
    #Finding teams and dividing into home and away teams
    all_teams_list = []
    for line in cricket_data:
        home_teams = line.find_all('p', class_='txt1')
        for team in home_teams:
            team = team.text.upper()
            team = team.split(' ')
            team = team[:-3]
            team = ' '.join(team)
            all_teams_list.append(team)
    home_team_list_prep = []
    home_team_list_prep.append(all_teams_list[::2])
    for item in home_team_list_prep:
        for element in item:
            home_team_list.append(element)
    away_team_list_prep = []
    away_team_list_prep.append(all_teams_list[1::2])
    for item in away_team_list_prep:
        for element in item:
            away_team_list.append(element)
    #Finding scores
    for line in cricket_data:
        scores = line.find_all('p', class_='txt2 hs')
        all_runs = []
        for score in scores:
            runs = score.text.strip()
            runs = runs.split('/')[0].strip()
            all_runs.append(runs)
        home_runs_prep = []
        home_runs_prep.append(all_runs[::2])
        for lst in home_runs_prep:
            for item in lst:
                home_team_score_list.append(item)
        away_runs_prep = []
        away_runs_prep.append(all_runs[1::2])
        for lst in away_runs_prep:
            for item in lst:
                away_team_score_list.append(item)
    #Finding result and year
    for line in cricket_data:
        results = line.find_all('div', class_ = 'fonts-gt')
        for result in results:
            result_list.append(result.text.upper())
            year = url.split('?')[1]
            year = year.split('=')[1]
            year = year.split('&')[0]
            years.append(year)
            win = result.text.split('WON')[0].strip()
            if win == 'ABANDONED':
                win = None
            elif win == 'CANCELLED':
                win = None
            elif 'CONCEDED' in win:
                win = None
            elif win == 'TIED':
                win = None
            else:
                win = win
            winner.append(win)
            margin = result.text.split(' ')[-1].strip()
            #print(margin)
            if margin == 'WICKETS':
                bat_first_win.append(False)
                bat_second_win.append(True)
            elif margin == 'WICKET':
                bat_first_win.append(False)
                bat_second_win.append(True)
            elif margin == 'RUN':
                bat_first_win.append(True)
                bat_second_win.append(False)
            elif margin == 'RUNS':
                bat_first_win.append(True)
                bat_second_win.append(False)
            elif margin == 'ABANDONED':
                bat_first_win.append(None)
                bat_second_win.append(None)
            elif margin == 'CANCELLED':
                bat_first_win.append(None)
                bat_second_win.append(None)
            elif margin == 'CONCEDED':
                bat_first_win.append(None)
                bat_second_win.append(None)
            elif margin == 'TIED':
                bat_first_win.append(True)
                bat_second_win.append(True)
    #Finding wickets taken
    for line in cricket_data:
        scores = line.find_all('p', class_ = 'txt2 hs')
        wickets_prep_list = []
        home_wickets_prep_list = []
        away_wickets_prep_list = []
        for score in scores:
            wickets = score.text.strip()
            wickets = wickets.split('/')
            if wickets != ['']:
                wickets = wickets[1].split('  ')[0].strip()
                if wickets == 'All out':
                    wickets = 10
            else:
                wickets = None
            wickets_prep_list.append(wickets)
    home_wickets_prep_list = wickets_prep_list[::2]
    for item in home_wickets_prep_list:
        home_team_wickets_lost_list.append(item)
    away_wickets_prep_list = wickets_prep_list[1::2]
    for item in away_wickets_prep_list:
        away_team_wickets_lost_list.append(item)
    print(f'{url} parsed')
    #print('proper len', len(home_team_score_list))
    #print('bat first win', len(bat_first_win))
    #print('bat second win', len(bat_second_win))
#print(len(home_team_score_list))
#print(len(away_team_score_list))
#print(len(home_team_list))
#print(len(away_team_list))
#print(len(result_list))
#print(len(home_team_wickets_lost_list))
#print(len(away_team_wickets_lost_list))
#print(len(years))
#print(len(winner))  
#print(len(bat_first_win))
#print(len(bat_second_win))
cricketdf = pd.DataFrame({'Year':years, 'Home_Team':home_team_list, 'Home_Score':home_team_score_list,
                          'Home_Team_wkts_lost':home_team_wickets_lost_list,
                          'Away_Team':away_team_list, 'Away_Score':away_team_score_list,
                          'Away_Team_wkts_lost':away_team_wickets_lost_list,
                          'Result':result_list, 'Winner':winner, 'Bat_First_Win':bat_first_win, 
                          'Bat_Second_Win':bat_second_win})
cricketdf['Home_Team_Win'] = np.where(cricketdf['Home_Team']==cricketdf['Winner'] , True, False)
cricketdf['Away_Team_Win'] = np.where(cricketdf['Away_Team']==cricketdf['Winner'], True, False)
cricketdf

https://audleyend.play-cricket.com/website/team_matches/71589?season=2017&tab=league parsed
https://ncissc.play-cricket.com/website/team_matches/9822?season=2017&tab=league parsed
https://gchishill.play-cricket.com/website/team_matches/63027?season=2017&tab=league parsed
https://lintonvillage.play-cricket.com/website/team_matches/9962?season=2017&tab=league parsed
https://ickleton.play-cricket.com/website/team_matches/165509?season=2017&tab=league parsed
https://birchanger.play-cricket.com/website/team_matches/85067?season=2017&tab=league parsed
https://elmdoncc.play-cricket.com/website/team_matches/51838?season=2017&tab=league parsed
https://littleshelford.play-cricket.com/website/team_matches/19931?season=2017&tab=league parsed
https://ashwell.play-cricket.com/website/team_matches/52157?season=2017&tab=league parsed
https://audleyend.play-cricket.com/website/team_matches/71589?season=2018&tab=league parsed
https://ncissc.play-cricket.com/website/team_matches/9822?season=2018&tab=leag

,Year,Home_Team,Home_Score,Home_Team_wkts_lost,Away_Team,Away_Score,Away_Team_wkts_lost,Result,Winner,Bat_First_Win,Bat_Second_Win,Home_Team_Win,Away_Team_Win
0,2017,AUDLEY END & LITTLEBURY CC,181,7,"BARRINGTON CC, CAMBS",180,8,AUDLEY END & LITTLEBURY CC WON BY 3 WICKETS,AUDLEY END & LITTLEBURY CC,False,True,True,False
1,2017,SAWSTON AND BABRAHAM CC,154,8,AUDLEY END & LITTLEBURY CC,155,9,AUDLEY END & LITTLEBURY CC WON BY 1 WICKET,AUDLEY END & LITTLEBURY CC,False,True,False,True
2,2017,AUDLEY END & LITTLEBURY CC,213,7,MADINGLEY CC,201,9,AUDLEY END & LITTLEBURY CC WON BY 12 RUNS,AUDLEY END & LITTLEBURY CC,True,False,True,False
3,2017,AUDLEY END & LITTLEBURY CC,208,9,YELLING CC,120,10,AUDLEY END & LITTLEBURY CC WON BY 88 RUNS,AUDLEY END & LITTLEBURY CC,True,False,True,False
4,2017,CHESTERFORDS CC,145,7,AUDLEY END & LITTLEBURY CC,147,2,AUDLEY END & LITTLEBURY CC WON BY 8 WICKETS,AUDLEY END & LITTLEBURY CC,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,2021,ASHWELL CC,212,3,GREAT CHISHILL CC,217,2,GREAT CHISHILL CC WON BY 8 WICKETS,GREAT CHISHILL CC,False,True,False,True
581,2021,ASHWELL CC,193,4,AUDLEY END & LITTLEBURY CC,114,10,ASHWELL CC WON BY 6 WICKETS,ASHWELL CC,False,True,True,False
582,2021,RICKLING RAMBLERS CC,110,0,ASHWELL CC,105,8,RICKLING RAMBLERS CC WON BY 10 WICKETS,RICKLING RAMBLERS CC,False,True,True,False
583,2021,ASHWELL CC,,None,WHITTLESFORD CC,,None,CANCELLED,None,None,None,False,False


In [195]:
cricketdf.loc[cricketdf['Result'] == 'TIED']

,Year,Home_Team,Home_Score,Home_Team_wkts_lost,Away_Team,Away_Score,Away_Team_wkts_lost,Result,Winner,Bat_First_Win,Bat_Second_Win,Home_Team_Win,Away_Team_Win
451,2021,AUDLEY END & LITTLEBURY CC,201,8,LINTON VILLAGE CC,201,10,TIED,None,True,True,False,False
473,2021,GREAT CHISHILL CC,231,3,LINTON VILLAGE CC,231,7,TIED,None,True,True,False,False
489,2021,GREAT CHISHILL CC,231,3,LINTON VILLAGE CC,231,7,TIED,None,True,True,False,False
499,2021,AUDLEY END & LITTLEBURY CC,201,8,LINTON VILLAGE CC,201,10,TIED,None,True,True,False,False


In [199]:
duplicatematches = cricketdf[cricketdf.duplicated(keep=False)]
duplicatematches.to_csv('duplicates.csv', index=False)

In [200]:
duplicatematches

,Year,Home_Team,Home_Score,Home_Team_wkts_lost,Away_Team,Away_Score,Away_Team_wkts_lost,Result,Winner,Bat_First_Win,Bat_Second_Win,Home_Team_Win,Away_Team_Win
6,2017,AUDLEY END & LITTLEBURY CC,100,3,ASHWELL CC,,None,ABANDONED,None,None,None,False,False
14,2017,ASHWELL CC,186,4,AUDLEY END & LITTLEBURY CC,184,10,ASHWELL CC WON BY 6 WICKETS,ASHWELL CC,False,True,True,False
52,2017,LINTON VILLAGE CC,65,2,ELMDON CC,59,10,LINTON VILLAGE CC WON BY 8 WICKETS,LINTON VILLAGE CC,False,True,True,False
62,2017,ELMDON CC,221,1,LINTON VILLAGE CC,222,3,LINTON VILLAGE CC WON BY 7 WICKETS,LINTON VILLAGE CC,False,True,False,True
100,2017,LINTON VILLAGE CC,65,2,ELMDON CC,59,10,LINTON VILLAGE CC WON BY 8 WICKETS,LINTON VILLAGE CC,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,2021,GREAT CHISHILL CC,198,3,ASHWELL CC,197,8,GREAT CHISHILL CC WON BY 7 WICKETS,GREAT CHISHILL CC,False,True,True,False
579,2021,LITTLE SHELFORD CC,210,6,ASHWELL CC,207,6,LITTLE SHELFORD CC WON BY 4 WICKETS,LITTLE SHELFORD CC,False,True,True,False
580,2021,ASHWELL CC,212,3,GREAT CHISHILL CC,217,2,GREAT CHISHILL CC WON BY 8 WICKETS,GREAT CHISHILL CC,False,True,False,True
581,2021,ASHWELL CC,193,4,AUDLEY END & LITTLEBURY CC,114,10,ASHWELL CC WON BY 6 WICKETS,ASHWELL CC,False,True,True,False


In [209]:
df = cricketdf.drop_duplicates()
df = df[df.Bat_First_Win.notnull()]
df

,Year,Home_Team,Home_Score,Home_Team_wkts_lost,Away_Team,Away_Score,Away_Team_wkts_lost,Result,Winner,Bat_First_Win,Bat_Second_Win,Home_Team_Win,Away_Team_Win
0,2017,AUDLEY END & LITTLEBURY CC,181,7,"BARRINGTON CC, CAMBS",180,8,AUDLEY END & LITTLEBURY CC WON BY 3 WICKETS,AUDLEY END & LITTLEBURY CC,False,True,True,False
1,2017,SAWSTON AND BABRAHAM CC,154,8,AUDLEY END & LITTLEBURY CC,155,9,AUDLEY END & LITTLEBURY CC WON BY 1 WICKET,AUDLEY END & LITTLEBURY CC,False,True,False,True
2,2017,AUDLEY END & LITTLEBURY CC,213,7,MADINGLEY CC,201,9,AUDLEY END & LITTLEBURY CC WON BY 12 RUNS,AUDLEY END & LITTLEBURY CC,True,False,True,False
3,2017,AUDLEY END & LITTLEBURY CC,208,9,YELLING CC,120,10,AUDLEY END & LITTLEBURY CC WON BY 88 RUNS,AUDLEY END & LITTLEBURY CC,True,False,True,False
4,2017,CHESTERFORDS CC,145,7,AUDLEY END & LITTLEBURY CC,147,2,AUDLEY END & LITTLEBURY CC WON BY 8 WICKETS,AUDLEY END & LITTLEBURY CC,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,2021,LITTLE SHELFORD CC,186,7,RICKLING RAMBLERS CC,190,9,RICKLING RAMBLERS CC WON BY 1 WICKET,RICKLING RAMBLERS CC,False,True,False,True
569,2021,ASHWELL CC,129,7,THRIPLOW CC,126,9,ASHWELL CC WON BY 3 WICKETS,ASHWELL CC,False,True,True,False
575,2021,WHITTLESFORD CC,190,4,ASHWELL CC,188,9,WHITTLESFORD CC WON BY 6 WICKETS,WHITTLESFORD CC,False,True,True,False
576,2021,THRIPLOW CC,140,5,ASHWELL CC,139,10,THRIPLOW CC WON BY 5 WICKETS,THRIPLOW CC,False,True,True,False


In [210]:
df.to_csv('cricket_data.csv', index = True)